# ক্লাউডে ডেটা সায়েন্স: "Azure ML SDK" পদ্ধতি

## ভূমিকা

এই নোটবুকে, আমরা শিখব কীভাবে Azure ML SDK ব্যবহার করে একটি মডেল প্রশিক্ষণ, স্থাপন এবং ব্যবহার করা যায়।

প্রয়োজনীয়তা:
1. আপনি একটি Azure ML ওয়ার্কস্পেস তৈরি করেছেন।
2. আপনি [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) Azure ML-এ লোড করেছেন।
3. আপনি এই নোটবুকটি Azure ML Studio-তে আপলোড করেছেন।

পরবর্তী ধাপগুলো:

1. একটি বিদ্যমান ওয়ার্কস্পেসে একটি এক্সপেরিমেন্ট তৈরি করুন।
2. একটি কম্পিউট ক্লাস্টার তৈরি করুন।
3. ডেটাসেট লোড করুন।
4. AutoMLConfig ব্যবহার করে AutoML কনফিগার করুন।
5. AutoML এক্সপেরিমেন্ট চালান।
6. ফলাফল বিশ্লেষণ করুন এবং সেরা মডেলটি পান।
7. সেরা মডেলটি রেজিস্টার করুন।
8. সেরা মডেলটি স্থাপন করুন।
9. এন্ডপয়েন্ট ব্যবহার করুন।

## Azure Machine Learning SDK-সম্পর্কিত ইমপোর্টসমূহ


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## ওয়ার্কস্পেস আরম্ভ করুন  
সংরক্ষিত কনফিগারেশন থেকে একটি ওয়ার্কস্পেস অবজেক্ট আরম্ভ করুন। নিশ্চিত করুন যে কনফিগারেশন ফাইলটি .\config.json এ উপস্থিত আছে।  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## একটি Azure ML পরীক্ষা তৈরি করুন

চলুন আমরা 'aml-experiment' নামে একটি পরীক্ষা তৈরি করি সেই কর্মক্ষেত্রে যা আমরা ঠিক এখনই আরম্ভ করেছি।


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## একটি কম্পিউট ক্লাস্টার তৈরি করুন  
আপনার AutoML রান-এর জন্য একটি [কম্পিউট টার্গেট](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) তৈরি করতে হবে।  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## ডেটা
নিশ্চিত করুন যে আপনি ডেটাসেটটি Azure ML-এ আপলোড করেছেন এবং যে কীটি ডেটাসেটের নামের সাথে একই।


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## অটোএমএল কনফিগারেশন


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## অটোএমএল রান


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## সেরা মডেলটি ডিপ্লয় করুন

সেরা মডেলটি ডিপ্লয় করতে নিচের কোডটি চালান। আপনি Azure ML পোর্টালে ডিপ্লয়মেন্টের অবস্থা দেখতে পারবেন। এই ধাপটি সম্পন্ন হতে কয়েক মিনিট সময় লাগতে পারে।


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## এন্ডপয়েন্ট ব্যবহার করুন
আপনি নিম্নলিখিত ইনপুট নমুনায় ইনপুট যোগ করতে পারেন।


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতা নিশ্চিত করার চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
